### Problem 1 (50 points) 

Vapor-liquid equilibria data are correlated using two adjustable parameters $A_{12}$ and $A_{21}$ per binary
mixture. For low pressures, the equilibrium relation can be formulated as:

$$
\begin{aligned}
p = & x_1\exp\left(A_{12}\left(\frac{A_{21}x_2}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{water}^{sat}\\
& + x_2\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{1,4 dioxane}^{sat}.
\end{aligned}
$$

Here the saturation pressures are given by the Antoine equation

$$
\log_{10}(p^{sat}) = a_1 - \frac{a_2}{T + a_3},
$$

where $T = 20$($^{\circ}{\rm C}$) and $a_{1,2,3}$ for a water - 1,4 dioxane
system is given below.

|             | $a_1$     | $a_2$      | $a_3$     |
|:------------|:--------|:---------|:--------|
| Water       | 8.07131 | 1730.63  | 233.426 |
| 1,4 dioxane | 7.43155 | 1554.679 | 240.337 |


The following table lists the measured data. Recall that in a binary system $x_1 + x_2 = 1$.

|$x_1$ | 0.0 | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
|:-----|:--------|:---------|:--------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
|$p$| 28.1 | 34.4 | 36.7 | 36.9 | 36.8 | 36.7 | 36.5 | 35.4 | 32.9 | 27.7 | 17.5 |

Estimate $A_{12}$ and $A_{21}$ using data from the above table: 

1. Formulate the least square problem; 
2. Since the model is nonlinear, the problem does not have an analytical solution. Therefore, solve it using the gradient descent or Newton's method implemented in HW1; 
3. Compare your optimized model with the data. Does your model fit well with the data?

---

### Problem 2 (50 points) 

Solve the following problem using Bayesian Optimization:
$$
    \min_{x_1, x_2} \quad \left(4-2.1x_1^2 + \frac{x_1^4}{3}\right)x_1^2 + x_1x_2 + \left(-4 + 4x_2^2\right)x_2^2,
$$
for $x_1 \in [-3,3]$ and $x_2 \in [-2,2]$. A tutorial on Bayesian Optimization can be found [here](https://thuijskens.github.io/2016/12/29/bayesian-optimisation/).





In [6]:
# calculating psat water and psat 1,4 dioxide
import math
import numpy as np
import torch 

from torch.autograd import Variable

def line_search(a, e):
    start_step = 1.  # initialize step size
    
    def phi(start_step,a,e):
        return obj(a)+start_step*0.8*np.dot(a.grad,e)

    while phi(start_step,a,e)<obj(a+start_step*e):  # while phi(a,x)<obj(x-a*grad(x)): # if f(x+a*d)>phi(a) then backtrack. d is the search direction
        start_step = 0.5*start_step
       # print("second step",start_step)
        
    return start_step



a = Variable(t.tensor([2.,1.]), requires_grad=True)

b=torch.tensor([[8.07131,1730.63,233.426],[7.43155,1554.679,240.337]])
#psat=torch.tensor([ ])
lm=[]
x1=torch.tensor([0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
pexp=torch.tensor([28.1,34.4,36.7,36.9,36.8,36.7,36.5,35.4,32.9,27.7,17.5])
loss=0
    
for i in range(2):
    l=b[i][1]/(20+b[0][2])
    m=b[i][0]-l
    d=[10**m]
    lm.append(d)

psat=torch.tensor(lm)

for i in range(len(x1)):
    x2=1-x1[i]
    l1=a[1]*x2/(a[0]*x1[i]+a[1]*x2)
    l1woexp=a[0]*(l1**2)    
    l1wexp=torch.exp(l1woexp)
    l1full=x1[i]*l1wexp*psat[0]
    r1=a[0]*x1[i]/(a[0]*x1[i]+a[1]*x2)
    r1woexp=a[1]*(l1**2)
    r1wexp=torch.exp(r1woexp)
    r1full=x1[i]*r1wexp*psat[1]  
    pcal=l1full+r1full           
    loss=loss+(pcal-pexp[i])**2

obj = lambda a:loss

loss.backward()

eps = 1e-3  


k = 0  

soln = torch.tensor(a) 
calnorm=a.grad



error = torch.linalg.norm(calnorm)
print("error",error)
                          
start_step = 0.01 

# Armijo line search


while error >= eps:  # keep searching while gradient norm is larger than eps
    
    e = -a.grad
    print("e",e)
    
    
    
    start_step = line_search(a, e)
    
    print("after loop", start_step)
    

    
    a = a+start_step*e
    
    print("second a",a)
    
    
    print("after",a.grad)                     #not able to calculate grad here     
    
    error = torch.linalg.norm(a.grad)
    print(error)

print(a.data.numpy())
print(loss.data.numpy())  

error tensor(1714.5715)
e tensor([ -18.5664, 1714.4711])
after loop 2.9103830456733704e-11
second a tensor([2., 1.], grad_fn=<AddBackward0>)
after None


<ipython-input-6-0148071551d5>:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  soln = torch.tensor(a)
<ipython-input-6-0148071551d5>:92: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  print("after",a.grad)                     #not able to calculate grad here
<ipython-input-6-0148071551d5>:94: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradie

TypeError: linalg_norm(): argument 'input' (position 1) must be Tensor, not NoneType

In [9]:
 #!pip install bayesian-optimization
    
from bayes_opt import BayesianOptimization
def fun(x0,x1):
    return -((4-(2.1*x0**2)+((x0**4)/3))*x0**2+(x0*x1)+(-4+(4*x1**2))*x1**2)
    from bayes_opt import BayesianOptimization
limits= {'x0': (-3,3), 'x1': (-2,2)}

optimizer= BayesianOptimization(f=fun, pbounds=limits, random_state=1)
optimizer.maximize(init_points=2,n_iter=50)
print(optimizer.max)

|   iter    |  target   |    x0     |    x1     |
-------------------------------------------------
|  1        |  0.265    | -0.4979   |  0.8813   |
|  2        | -110.1    | -2.999    | -0.7907   |
|  3        | -0.4933   | -0.3849   |  1.039    |
|  4        | -0.2833   |  1.599    | -0.5696   |
|  5        | -162.9    |  3.0      |  2.0      |
|  6        | -47.77    |  0.3665   | -2.0      |
|  7        | -150.9    |  3.0      | -2.0      |
|  8        | -0.7638   |  0.4683   | -0.02769  |
|  9        | -48.27    | -2.043    |  2.0      |
|  10       | -2.236    |  1.314    |  0.6422   |
|  11       | -50.26    |  0.5766   |  2.0      |
|  12       | -52.97    | -1.299    | -2.0      |
|  13       | -1.06     | -0.7025   | -0.5363   |
|  14       | -7.993    |  2.197    |  0.07143  |
|  15       | -48.19    | -0.8016   |  2.0      |
|  16       | -1.426    | -1.507    |  0.2863   |
|  17       | -107.6    | -3.0      |  1.181    |
|  18       | -2.305    |  1.361    | -0.01255  |
